Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])


Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))


In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 2.954511
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.581391
Minibatch accuracy: 25.0%
Validation accuracy: 53.1%
Minibatch loss at step 100: 1.257633
Minibatch accuracy: 50.0%
Validation accuracy: 66.6%
Minibatch loss at step 150: 0.844607
Minibatch accuracy: 75.0%
Validation accuracy: 77.7%
Minibatch loss at step 200: 0.893772
Minibatch accuracy: 75.0%
Validation accuracy: 77.7%
Minibatch loss at step 250: 0.655003
Minibatch accuracy: 87.5%
Validation accuracy: 78.0%
Minibatch loss at step 300: 0.683066
Minibatch accuracy: 75.0%
Validation accuracy: 79.7%
Minibatch loss at step 350: 0.446182
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 400: 0.393321
Minibatch accuracy: 93.8%
Validation accuracy: 77.7%
Minibatch loss at step 450: 0.786982
Minibatch accuracy: 81.2%
Validation accuracy: 79.7%
Minibatch loss at step 500: 0.543800
Minibatch accuracy: 87.5%
Validation accuracy: 81.6%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [5]:
### Define your architecture here.

# Hyper parameters
batch_size = 128
image_size = 28
num_channels = 1
input = image_size*image_size*num_channels
patch_size = 3
depth = 16
f1_out = 1024
f2_out = 512
num_labels = 10
wd_coeff = 0.0
drop_out1 = 0.8
drop_out2 = 0.5
drop_out3 = 0.5
initial_learning_rate = 1e-4

f1_in = image_size // 4 * image_size // 4 * (depth * 2)
#f1_in = 1568

# Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size, image_size, num_channels))
tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
tf_valid_dataset = tf.constant(valid_dataset)
tf_test_dataset = tf.constant(test_dataset)

init_conv2d = tf.contrib.layers.xavier_initializer_conv2d(uniform=True, seed=None, dtype=tf.float32)
init = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32)
weights = {
           'c1': tf.get_variable("c1", shape=[patch_size, patch_size, num_channels, depth], initializer=init_conv2d),
           'c2': tf.get_variable("c2", shape=[patch_size, patch_size, depth, depth * 2], initializer=init_conv2d),
#           'c3': tf.get_variable("c3", shape=[patch_size, patch_size, depth * 2, depth * 4], initializer=init_conv2d),
#           'c4': tf.get_variable("c4", shape=[patch_size, patch_size, depth * 4, depth * 4], initializer=init_conv2d),
           'f1': tf.get_variable("f1", shape=[f1_in, f1_out], initializer=init),
           'f2': tf.get_variable("f2", shape=[f1_out, f2_out], initializer=init),
           'out': tf.get_variable("out", shape=[f2_out, num_labels], initializer=init)}

bias = {
          'c1': tf.Variable(tf.zeros([depth])),
          'c2': tf.Variable(tf.zeros([depth*2])),
#          'c3': tf.Variable(tf.zeros([depth*4])),
#          'c4': tf.Variable(tf.zeros([depth*4])),
          'f1': tf.Variable(tf.zeros([f1_out])),
          'f2': tf.Variable(tf.zeros([f2_out])),
          'out': tf.Variable(tf.zeros([num_labels]))}

regularizers = (tf.nn.l2_loss(weights['c1']) + tf.nn.l2_loss(bias['c1']) +
              tf.nn.l2_loss(weights['c2']) + tf.nn.l2_loss(bias['c2']) +
#              tf.nn.l2_loss(weights['c3']) + tf.nn.l2_loss(bias['c3']) +
#              tf.nn.l2_loss(weights['c4']) + tf.nn.l2_loss(bias['c4']) +
              tf.nn.l2_loss(weights['f1']) + tf.nn.l2_loss(bias['f1']) +
              tf.nn.l2_loss(weights['f2']) + tf.nn.l2_loss(bias['f2']) +
              tf.nn.l2_loss(weights['out']) + tf.nn.l2_loss(bias['out']))

keep_prob1 = tf.placeholder(tf.float32)
keep_prob2 = tf.placeholder(tf.float32)
keep_prob3 = tf.placeholder(tf.float32)


In [6]:
from tensorflow.contrib.layers import flatten

def convolve(x, W, b, strides, pad):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding=pad)
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def Model(x, use_dropout=False):    
                                          
    # Layer 1: Convolutional. 
    conv1 = convolve(x, weights['c1'], bias['c1'], 2, 'SAME')
    
    # Layer 2: Convolutional. 
    conv2 = convolve(conv1, weights['c2'], bias['c2'], 1, 'SAME')
    conv2= tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

    if use_dropout:
        conv2 = tf.nn.dropout(conv2, keep_prob1)        
    #conv2 = tf.nn.batch_normalization(conv2, mean=0.0, variance=1.0, offset=None, scale=None, variance_epsilon=0.001)

    # Layer 3: Convolutional. 
#    conv3 = convolve(conv2, weights['c3'], bias['c3'], 1, 'SAME')

    # Layer 4: Convolutional. 
#    conv4 = convolve(conv3, weights['c4'], bias['c4'], 1, 'SAME')
#    conv4 = tf.nn.max_pool(conv4, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    #conv4 = tf.nn.batch_normalization(conv4, mean=0.0, variance=1.0, offset=None, scale=None, variance_epsilon=0.001)    

    # Flatten.
    reshape = flatten(conv2)

    # Layer 5: Fully Connected. 
    if use_dropout:
        reshape = tf.nn.dropout(reshape, keep_prob1)    
    h_layer1 = tf.matmul(reshape, weights['f1']) + bias['f1']
    h_layer1 = tf.nn.relu(h_layer1)
    #h_layer1 = tf.nn.batch_normalization(h_layer1, mean=0.0, variance=1.0, offset=None, scale=None, variance_epsilon=0.001)
    
    # Layer 6: Fully Connected. 
    if use_dropout:
        h_layer1 = tf.nn.dropout(h_layer1, keep_prob2)
    h_layer2 = tf.matmul(h_layer1, weights['f2']) + bias['f2']
    h_layer2 = tf.nn.relu(h_layer2)
    #h_layer2 = tf.nn.batch_normalization(h_layer2, mean=0.0, variance=1.0, offset=None, scale=None, variance_epsilon=0.001)
    
    # Layer 7: Fully Connected. 
    if use_dropout:
        h_layer2 = tf.nn.dropout(h_layer2, keep_prob3)
    logits = tf.matmul(h_layer2, weights['out']) + bias['out']

    return logits


In [7]:

logits = Model(tf_train_dataset, True)

classification_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

weightdecay_loss = wd_coeff * regularizers
loss = classification_loss + weightdecay_loss
    
# Optimizer.
global_step = tf.Variable(0)
lr = tf.train.exponential_decay(initial_learning_rate, global_step, 3500, 0.98, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate = lr)
#training_operation = optimizer.minimize(loss)
training_operation = optimizer.minimize(loss, global_step=global_step)
  
# Predictions for the training, validation, and test data.
train_prediction = tf.nn.softmax(logits)

valid_logits = Model(tf_valid_dataset)
valid_prediction = tf.nn.softmax(valid_logits)

test_logits = Model(tf_test_dataset)
test_prediction = tf.nn.softmax(test_logits)


In [10]:
EPOCHS = 8
num_steps = EPOCHS * 3500
from sklearn.utils import shuffle

with tf.Session() as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):

    if (step % 3500 == 0):
        train_dataset, train_labels = shuffle(train_dataset, train_labels)
    
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob1: drop_out1, keep_prob2: drop_out2, keep_prob3: drop_out3}

    _, l, predictions = session.run([training_operation, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 2000 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Initialized
Minibatch loss at step 0: 2.288231
Minibatch accuracy: 11.7%
Validation accuracy: 10.5%
Minibatch loss at step 2000: 0.449171
Minibatch accuracy: 85.9%
Validation accuracy: 87.2%
Minibatch loss at step 4000: 0.625508
Minibatch accuracy: 82.0%
Validation accuracy: 88.9%
Minibatch loss at step 6000: 0.350826
Minibatch accuracy: 86.7%
Validation accuracy: 90.2%
Minibatch loss at step 8000: 0.299814
Minibatch accuracy: 89.8%
Validation accuracy: 90.7%
Minibatch loss at step 10000: 0.306230
Minibatch accuracy: 92.2%
Validation accuracy: 91.1%
Minibatch loss at step 12000: 0.443307
Minibatch accuracy: 86.7%
Validation accuracy: 91.5%
Minibatch loss at step 14000: 0.157784
Minibatch accuracy: 96.1%
Validation accuracy: 91.6%
Minibatch loss at step 16000: 0.318861
Minibatch accuracy: 88.3%
Validation accuracy: 91.8%
Minibatch loss at step 18000: 0.391344
Minibatch accuracy: 88.3%
Validation accuracy: 92.1%
Minibatch loss at step 20000: 0.242087
Minibatch accuracy: 92.2%
Validation 